# MongoDB/Pymongo Indexes and Text Search

Work in driver/navigator pairs with a single laptop. Talk through each idea before you code so both partners understand the plan.

## 1. Setup

Import the core libraries we will need for HTTP requests, JSON inspection, and quick analyses.

In [ ]:
from pymongo import MongoClient
from pprint import pprint

# 2. Retrieve posts from your local MongoDB database

In [ ]:
# create database connection
client = MongoClient()
db = client.mastodon_test
db

In [ ]:
# show collections in mastodon_test
db.list_collection_names()

In [ ]:
# assign variable to collection in mastodon_test
coll = ...
coll

In [ ]:
# confirm number of documents
coll...

In [ ]:
# show a sample post from the collection
sample = ...
pprint(sample)

# 3. .explain()

In [ ]:
# Start by using explain to see what MongoDB does in a basic .find() query.
exp = coll.find({}).explain()
pprint(exp)

Copy this output and paste it into ChatGPT, Claude, or Gemini, and ask it to explain what it sees in plain English, noting any indexes it sees and is using.

Things are funky in Pymongo when we're dealing with aggregation queries, though. _More complicated than in your reading on MongoDB!_ Let's unpack this with a pipeline aggregation query from last week's notebook.

In [ ]:
pipeline = [
    {'$unwind': '$tags'},
    {'$project': {
        '_id': 0,
        'acct': '$account.acct',
        'created_at': 1,
        'hashtag': '$tags.name'
    }},
    {'$group': {
        '_id': '$hashtag',
        'count': {'$sum': 1}
    }},
    {'$sort': {'count': -1}},
    {'$limit': 10}
]

result = coll.aggregate(pipeline)
for doc in result:
    pprint(doc)

If we do what we did above with the .find() query, it won't work.

In [ ]:
exp = coll.aggregate(pipeline).explain()
pprint(exp)

Here's how we have to do it. (Hang onto this code so you can reuse it in the future.)

In [ ]:
plan = coll.database.command(
    "explain",
    {
        "aggregate": coll.name,
        "pipeline": pipeline,
        "cursor": {}
    },
    verbosity="executionStats"  # ✅ accepted here
)

pprint(plan)

Now copy that output, paste it into an AI chat, and ask it to explain it in plain English, specifically referencing any indexes it finds.

In [ ]:
# now take ANY .aggregate() query from last week's notebook, run .explain() on it, and ask an AI chat to explain it in plain English.

...

## 4. Standard indexes

Let's practice indexing with the account.acct field for faster account searching.

In [ ]:
# Pick a username from your database

username = coll.find_one()['account']['acct']
print(username)

In [ ]:
# search for all posts from that user and run .explain()
# Use AI (or your own eyes -- look at 'winningPlan', do you see 'COLLSCAN'?) to ensure that it is not using any indexes

posts = ...
pprint(posts.explain())

In [ ]:
# now let's index that field

coll.create_index([("account.acct", 1)],
                  name="acct_index") # name optional, but can help debug

In [ ]:
# rerun your explain and explore the output to ensure the index is being used
# use AI to help analyze your output if necessary (or just look for 'winningPlan' -- do you see an index? named acct_index?)

posts = ...
pprint(posts.explain())

Now let's do one on followers_count. Since we'll typically be looking for accounts with the MOST followers, we'll want the index in DESCENDING order.

In [ ]:
# first, run a find query that identifies the 10 posts (not accounts!) with the most followers, and add .explain().
# what method is it using?

exp = coll.find({}).sort('followers_count', -1).limit(10).explain()

pprint(exp)

In [ ]:
# now add a DESCENDING index on that field

...

In [ ]:
# rerun the explain to ensure the index is being used

exp = coll.find({}).sort('followers_count', -1).limit(10).explain()

pprint(exp)

## 5. Array indexes

To index every element in an array (like tags or media_attachments), just create an index on that field. MongoDB will automatically index each element in the arrays. Note, though, that these indexes will be more "expensive" (take up more space, add more time when adding new documents) than one that indexes one element per document (like account.acct or content).

In [ ]:
# grab the grouping query from last week's notebook that returns the top 10 hashtags (there were two, either one is fine). Run just that query.

pipeline = [
    {'$unwind': '$tags'},
    {'$group': {
        '_id': '$tags.name',
        'count': {'$sum': 1}
    }},
    {'$sort': {'count': -1}},
    {'$limit': 10}
]

result = coll.aggregate(pipeline)
for doc in result:
    pprint(doc)

In [ ]:
# now run explain() on this query (remember that aggregate requires a more involved method, but you can just copy and paste from above).
# This output will be more complicated, so you may want to use AI to help explain it to you and check for index usage.

plan = ...

pprint(plan)

In [ ]:
# now add an index to the hashtag name field, ascending is fine.

...

In [ ]:
# now rerun the above explain to ensure the index is being used.

plan = ...

pprint(plan)

In [ ]:
# let's check a simple hashtag search to see if it uses the index

coll.find({'tags.name': 'nature'}).explain()

## 6. Text indexes and searches

To perform a keyword or partial text search (like SQL's `WHERE content like '%keyword%'`), we need to use `$regex`.

In [ ]:
# perform a search for the word 'nature' somewhere in the post content

cursor = coll.find({'content': { '$regex': 'nature'}})

for post in cursor:
    pprint(post['content'])

In [ ]:
# add `$options: 'i'` to make it case-insensitive

cursor = coll.find({'content': { '$regex': 'nature',
                                 '$options': 'i'}})

for post in cursor:
    pprint(post['content'])

For the most robust text searches in MongoDB, we'd have to use their Atlas platform. Most significant is that Atlas is required to do partial-word matches with wildcards. However, as long as we're doing full-word searches, we can run that locally, and we can create a text index to support it.

In [ ]:
# first, run the above search with .explain() to see it without an index

exp = coll.find({'content': { '$regex': 'nature',
                              '$options': 'i'}})

pprint(exp.explain())

In [ ]:
# now create a TEXT index on content
# note the import statement -- you only need to do that once
# Also note the default_language and language_override fields. This is because Mastodon includes a language field, but it's not always populated, so MongoDB gets confused trying to decide what language(s) to use to support some of their advanced language parsing features. Just use this code if you want to avoid that problem with this data.

from pymongo import TEXT

coll.create_index([("content", TEXT)],
                  name="content_text_index",
                  default_language="english",
                  language_override="__no_override__"
                  )

In [ ]:
# now rerun the above keyword search to ensure the index is being used

exp = coll.find({'content': { '$regex': 'nature'}})

pprint(exp.explain())

In [ ]:
# $regex doesn't work with the TEXT index we created. We have to do the text search a little differently. (This method ONLY works with a TEXT index.)
# Again, just copy and paste this code, modifying as necessary, for future projects, including the midterm.

cursor = coll.find(
    {"$text": {"$search": "nature"}},
    {"content": 1}
)

pprint(cursor.explain())

In [ ]:
# Now let's use it in an aggregation pipeline.
# Perform the same search as a $match, then return the three matching posts with the highest followers_count.
# Return only the content and followers_count fields.

pipeline = [
    ...
]

cursor = coll.aggregate(pipeline)

for doc in cursor:
    pprint(doc)

In [ ]:
# Now run that same pipeline with explain to see if it's using the text index.
# By the way, ... is it using the followers_count index? Why or why not?

plan = ...

pprint(plan)

# Save this file WITH ALL OUTPUT SHOWING and submit to Canvas